In [0]:
!apt-get install protobuf-compiler python-pil python-lxml python-tk
!pip install Cython
!pip install jupyter
!pip install matplotlib

In [0]:
! pip3 install 'prompt-toolkit<2.0.0,>=1.0.15' --force-reinstall

In [0]:
!git clone https://github.com/tensorflow/models.git


In [0]:
cd /content/models/research/

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

In [0]:
%set_env PYTHONPATH=/content/models/research:/content/models/research/slim


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
cd  /content/gdrive/My\ Drive/Colab\ Notebooks

In [0]:
ll

In [0]:
#! wget http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz

In [0]:
#! tar xvzf ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz

In [0]:
! cp final_train_all.tfrecord /content/models/research/object_detection/data/

In [0]:
! cp final_test_all.tfrecord /content/models/research/object_detection/data/

In [0]:
! cp object-detection_test.pbtxt /content/models/research/object_detection/data/

In [0]:
! mkdir /content/models/research/object_detection/models/model

In [0]:
! cp ssdlite_mobilenet_v2_coco.config /content/models/research/object_detection/models/model/

In [0]:
! cp -r check /content/models/research/object_detection/models/model/

In [0]:
cd /content/models/research/object_detection/models/model

In [0]:
ls /content/models/research/object_detection/data/

In [0]:
ll

In [0]:
cd /content/models/research/

In [0]:
PYTHONPATH="$PYTHONPATH:/content/models/research:/content/models/research/slim" 

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'


In [0]:
! python object_detection/model_main.py \
--pipeline_config_path=/content/models/research/object_detection/models/model/ssdlite_mobilenet_v2_coco.config \
--model_dir=/content/gdrive/My\ Drive/Colab\ Notebooks/results \
--num_train_steps=100000 --sample_1_of_n_eval_examples=1 --alsologtostderr

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/device:GPU:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Ensure that TF can detect the GPU.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

In [0]:
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [0]:
! unzip ngrok-stable-linux-amd64.zip


In [0]:
get_ipython().system_raw('! tensorboard --logdir /content/gdrive/My\ Drive/Colab\ Notebooks/results --host 0.0.0.0 --port 6005 &')

In [0]:
! tensorboard --logdir /content/gdrive/My\ Drive/Colab\ Notebooks/results --host 0.0.0.0 --port 6005 &

In [0]:
get_ipython().system_raw('./ngrok http 6005 &')


In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
 "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
get_ipython().system_raw(
    'tensorboard --logdir /content/gdrive/My\ Drive/Colab\ Notebooks/results  --host 0.0.0.0 --port 6006 &')

! wget -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -o ngrok-stable-linux-amd64.zip

get_ipython().system_raw('./ngrok http 6006 &')

! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
! ls /content/gdrive/My\ Drive/Colab\ Notebooks/results